In [0]:
# Colab library to upload files to notebook
from google.colab import files

# Install Kaggle library
!pip install -q kaggle

# Upload kaggle API key file
uploaded = files.upload()
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6
!kaggle -v
!kaggle competitions download -c dogs-vs-cats

In [0]:
!unzip \*.zip

In [0]:
!unzip \*.zip

In [0]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from keras.preprocessing.image import ImageDataGenerator, load_img

categories = []
files = os.listdir('/content/train/')
for f in files:
    category = f.split('.')[0]
    if category == 'dog':
        categories.append(1)
    else:
        categories.append(0)

df = pd.DataFrame({
    'filename': files,
    'category': categories
})
df["category"] = df["category"].replace({0: 'cat', 1: 'dog'}) 
train_df, validate_df = train_test_split(df, test_size=0.20, random_state=42)
train_df = train_df.reset_index(drop=True)
validate_df = validate_df.reset_index(drop=True)

In [0]:
import keras
import tensorflow
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization,Conv2D,MaxPool2D,MaxPooling2D
import numpy as np
from sklearn.model_selection import train_test_split
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size=15
IMAGE_SIZE = (128,128)
IMAGE_CHANNELS = 3

In [57]:
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    "/content/train/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    "/content/train/", 
    x_col='filename',
    y_col='category',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=batch_size
)

model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) # 2 because we have cat and dog classes

model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


history = model.fit_generator(
    train_generator, 
    epochs=10,
    validation_data=validation_generator,
    validation_steps=total_validate//batch_size,
    steps_per_epoch=total_train//batch_size
)

Found 20000 validated image filenames belonging to 2 classes.
Found 5000 validated image filenames belonging to 2 classes.
Epoch 1/10
1333/1333 [==============================] - 168s 126ms/step - loss: 0.7512 - accuracy: 0.6268 - val_loss: 0.5517 - val_accuracy: 0.7119
Epoch 2/10
1333/1333 [==============================] - 162s 121ms/step - loss: 0.5627 - accuracy: 0.7162 - val_loss: 0.8758 - val_accuracy: 0.6209
Epoch 3/10
1333/1333 [==============================] - 155s 117ms/step - loss: 0.5069 - accuracy: 0.7569 - val_loss: 0.4197 - val_accuracy: 0.7597
Epoch 4/10
1333/1333 [==============================] - 157s 118ms/step - loss: 0.4741 - accuracy: 0.7794 - val_loss: 0.3357 - val_accuracy: 0.7669
Epoch 5/10
1333/1333 [==============================] - 155s 116ms/step - loss: 0.4444 - accuracy: 0.7964 - val_loss: 0.9783 - val_accuracy: 0.7145
Epoch 6/10
1333/1333 [==============================] - 156s 117ms/step - loss: 0.4194 - accuracy: 0.8130 - val_loss: 0.1669 - val_accura